Varroa Mite Detection V1.0

In [1]:
# Cell Block 1: Importing Libraries
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras import layers, models

In [2]:
# Cell Block 2: Load Data and Preprocess

df = pd.read_csv('labels.csv')

df['filename'] = df['filename'].apply(lambda fn: os.path.join('images', fn.strip()))

filepaths = df['filename'].values
labels = df['has_mite'].values

In [3]:
# Cell Block 3: K-Fold Cross-Validation
from tensorflow.keras.callbacks import EarlyStopping

# # define it once (you can tweak patience, monitor, etc. as you like)
# earlystop = EarlyStopping(
#     monitor='val_loss',        # watch validation loss
#     patience=5,               # stop if no improvement for 10 epochs
#     restore_best_weights=True  # roll back to the best weights seen
# )


k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold = 1
all_test_accuracies = []

for trainval_index, test_index in kf.split(filepaths):
    print(f"\n🧪 Fold {fold} -----------------------------")

    # Split into trainval and test
    X_trainval, X_test = filepaths[trainval_index], filepaths[test_index]
    y_trainval, y_test = labels[trainval_index], labels[test_index]

    # Further split trainval into train and val (e.g. 80/20)
    val_split = int(0.8 * len(X_trainval))
    X_train, X_val = X_trainval[:val_split], X_trainval[val_split:]
    y_train, y_val = y_trainval[:val_split], y_trainval[val_split:]


    def preprocess(paths, labels):
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))

        def load_img(path, label):
            img = tf.io.read_file(path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, [224, 224])
            img = img / 255.0
            return img, label

        return ds.map(load_img).batch(32).prefetch(tf.data.AUTOTUNE)

    train_ds = preprocess(X_train, y_train)
    val_ds = preprocess(X_val, y_val)
    test_ds = preprocess(X_test, y_test)


    # Build a fresh model for each fold
    def create_model():
        model = models.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Conv2D(32, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(128, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    model = create_model()
    # # Train
    # model.fit(train_ds, validation_data=val_ds, epochs=100, verbose=1)

    # Train with early stopping
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,            # generous upper bound
        # callbacks=[earlystop], # ← here!
        verbose=1
    )


    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_ds)
    all_test_accuracies.append(test_acc)
    print(f"✅ Fold {fold} test accuracy: {test_acc:.4f}")


    fold += 1





🧪 Fold 1 -----------------------------
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 432ms/step - accuracy: 0.5624 - loss: 1.3326 - val_accuracy: 0.5705 - val_loss: 0.6899
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 410ms/step - accuracy: 0.6297 - loss: 0.6707 - val_accuracy: 0.5705 - val_loss: 0.6832
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 407ms/step - accuracy: 0.6297 - loss: 0.6709 - val_accuracy: 0.5705 - val_loss: 0.6938
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 413ms/step - accuracy: 0.6297 - loss: 0.6711 - val_accuracy: 0.5705 - val_loss: 0.6834
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 403ms/step - accuracy: 0.6297 - loss: 0.6743 - val_accuracy: 0.5705 - val_loss: 0.6832
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 405ms/step - accuracy: 0.6297 - loss: 0.6659 - val_accuracy: 0.5705 - val_loss: 0.6850
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 412ms/step - accuracy: 0.6297 - loss: 0.6628 - val_accuracy: 0.5705 - val_loss: 0.6835
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 404ms/step - accuracy: 

In [5]:
# Cell Block 4: Final Evaluation
mean_acc = np.mean(all_test_accuracies)
std_acc = np.std(all_test_accuracies)
print(f"\n📊 {k}-Fold Test Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")


📊 10-Fold Test Accuracy: 0.6571 ± 0.0766
